In [1]:
import pandas as pd
import numpy as np
import sys
import os

import mysql.connector
from mysql.connector import Error

## SQL - fetch data

In [2]:
def connect_to_database(database, host='localhost', user='root', password='password', v=True):
    try:
        connection = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        
        if connection.is_connected():
            db_info = connection.get_server_info()
            if v:
                print("Connected to MySQL Server version", db_info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            if v:
                print("You're connected to database:", record)
            return connection, cursor

    except Error as e:
        if v: 
            print("Error while connecting to MySQL", e)

    return None


In [3]:
def close_connection(connection, cursor):
    if cursor:
        cursor.close()
    if connection and connection.is_connected():
        connection.close()
        print("MySQL connection is closed")

In [4]:
con, cur = connect_to_database('flight_prediction')

Connected to MySQL Server version 8.3.0
You're connected to database: ('flight_prediction',)


In [5]:
def execute_query(connection, cursor, query, params=None):
    try:
        cursor.execute(query, params)
        if query.strip().lower().startswith('select') or query.strip().lower().startswith('show'):
            result = cursor.fetchall()
            return result
        else:
            connection.commit()
            return cursor.rowcount
    except Error as e:
        print("Error executing query:", e)
        return None

In [6]:
def put_res(results, query_type="SELECT"):
    if not results:
        print("No results found.")
        return

    if query_type.lower() in ["select", "show"]:
        for row in results:
            print(row)
    else:
        print(f"Affected rows: {results}")

In [7]:
res = execute_query(con, cur, 'SHOW TABLES', params=None)
put_res(res)

('airline',)
('airport',)
('delay',)
('flight',)
('states',)


In [10]:
res = execute_query(con, cur, 'select * from flight join airport on origin_airport_seq_ID=airport_seq_ID join states on airport.state_ID=states.abr limit 10;', params=None)
put_res(res)

(1, datetime.datetime(2016, 1, 6, 0, 0), 19805, 'N4YBAA', 43, 1129804, 1143302, 1100, 1438, 158, 986, 4, 11298, 1129804, 30194, 'Dallas/Fort Worth, TX', 'TX', 'TX', 48, 'Texas', 74)
(2, datetime.datetime(2016, 1, 7, 0, 0), 19805, 'N434AA', 43, 1129804, 1143302, 1100, 1438, 158, 986, 4, 11298, 1129804, 30194, 'Dallas/Fort Worth, TX', 'TX', 'TX', 48, 'Texas', 74)
(3, datetime.datetime(2016, 1, 8, 0, 0), 19805, 'N541AA', 43, 1129804, 1143302, 1100, 1438, 158, 986, 4, 11298, 1129804, 30194, 'Dallas/Fort Worth, TX', 'TX', 'TX', 48, 'Texas', 74)
(4, datetime.datetime(2016, 1, 9, 0, 0), 19805, 'N489AA', 43, 1129804, 1143302, 1100, 1438, 158, 986, 4, 11298, 1129804, 30194, 'Dallas/Fort Worth, TX', 'TX', 'TX', 48, 'Texas', 74)
(5, datetime.datetime(2016, 1, 10, 0, 0), 19805, 'N439AA', 43, 1129804, 1143302, 1100, 1438, 158, 986, 4, 11298, 1129804, 30194, 'Dallas/Fort Worth, TX', 'TX', 'TX', 48, 'Texas', 74)
(6, datetime.datetime(2016, 1, 11, 0, 0), 19805, 'N468AA', 43, 1129804, 1143302, 1100, 14

In [11]:
def fetch_data_to_dataframe(connection, query):
    cursor = None
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        res = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(res, columns=columns)
        return df
    except Error as e:
        print(f"Error fetching data: {e}")
        return None
    finally:
        if cursor:
            cursor.close()

In [18]:
query = '''
SELECT 
    f.*, 
    ao.airport_ID AS origin_airport_ID
    ao.airport_seq_ID AS origin_airport_seq_ID
    ao.city_market_ID AS origin_city_market_ID
    ao.city AS origin_city
    
    so.abr AS origin_state_abr,
    so.FIPS AS origin_state_FIPS,
    so.name AS origin_state_name,
    so.WAC AS origin_state_WAC,
    
    ad.airport_ID AS dest_airport_ID
    ad.airport_seq_ID AS dest_airport_seq_ID
    ad.city_market_ID AS dest_city_market_ID
    ad.city AS dest_city
    
    sd.abr AS dest_state_abr,
    sd.FIPS AS dest_state_FIPS,
    sd.name AS dest_state_name,
    sd.WAC AS dest_state_WAC,
FROM 
    flight f
LEFT JOIN 
    airport ao ON f.origin_airport_seq_ID = ao.airport_seq_ID
LEFT JOIN 
    states so ON ao.state_ID = so.abr
LEFT JOIN 
    airport ad ON f.dest_airport_seq_ID = ad.airport_seq_ID
LEFT JOIN 
    states sd ON ad.state_ID = sd.abr;limit 10
'''
df = fetch_data_to_dataframe(con, query)

Error fetching data: MySQL Connection not available.


In [17]:
df

,ID,date,airline_ID,tail_num,fl_num,origin_airport_seq_ID,dest_airport_seq_ID,dep_time,arr_time,duration,...,airport_ID,airport_seq_ID,city_market_ID,city,state_ID,airport_ID,airport_seq_ID,city_market_ID,city,state_ID
0,1,2016-01-06,19805,N4YBAA,43,1129804,1143302,1100,1438,158,...,11298,1129804,30194,"Dallas/Fort Worth, TX",TX,11433,1143302,31295,"Detroit, MI",MI
1,2,2016-01-07,19805,N434AA,43,1129804,1143302,1100,1438,158,...,11298,1129804,30194,"Dallas/Fort Worth, TX",TX,11433,1143302,31295,"Detroit, MI",MI
2,3,2016-01-08,19805,N541AA,43,1129804,1143302,1100,1438,158,...,11298,1129804,30194,"Dallas/Fort Worth, TX",TX,11433,1143302,31295,"Detroit, MI",MI
3,4,2016-01-09,19805,N489AA,43,1129804,1143302,1100,1438,158,...,11298,1129804,30194,"Dallas/Fort Worth, TX",TX,11433,1143302,31295,"Detroit, MI",MI
4,5,2016-01-10,19805,N439AA,43,1129804,1143302,1100,1438,158,...,11298,1129804,30194,"Dallas/Fort Worth, TX",TX,11433,1143302,31295,"Detroit, MI",MI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5635962,5635963,2016-12-31,19393,N7703A,1077,1537602,1289204,755,830,95,...,15376,1537602,30436,"Tucson, AZ",AZ,12892,1289204,32575,"Los Angeles, CA",CA
5635963,5635964,2016-12-31,19393,N7815L,1345,1537602,1289204,1320,1355,95,...,15376,1537602,30436,"Tucson, AZ",AZ,12892,1289204,32575,"Los Angeles, CA",CA
5635964,5635965,2016-12-31,19393,N967WN,1176,1537602,1323202,705,1125,200,...,15376,1537602,30436,"Tucson, AZ",AZ,13232,1323202,30977,"Chicago, IL",IL
5635965,5635966,2016-12-31,19393,N271LV,865,1537602,1467903,1220,1235,75,...,15376,1537602,30436,"Tucson, AZ",AZ,14679,1467903,33570,"San Diego, CA",CA
